## Exploration of Cat Images




In [26]:
# used to import YOLO used for cropping provided imgs
from ultralytics import solutions
import cv2
from ultralytics import YOLO
# used to resume training if anything is interupted. Commeted out
# while learning so no test_test data persists
# results = model.train(resume=True)

# used for parsing files given their directory
import os
import shutil 

# used to encode cropped imgs into a vector of predictable length and size
import clip
import torch
from PIL import Image
import torchvision.transforms as transforms
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, clip_preprocess = clip.load('RN50', device=device)

from sklearn.model_selection import train_test_split

cat_names = ['mousse','sherbert']
TRAIN_TXT_PATH = f'C:\\Users\\kiki\\Desktop\\training\\data\\train\\labels\\'
TRAIN_IMAGE_PATH = f'C:\\Users\\kiki\\Desktop\\training\\data\\train\\images\\'
VAL_TXT_PATH = f'C:\\Users\\kiki\\Desktop\\training\\data\\val\\labels\\'
VAL_IMAGE_PATH = f'C:\\Users\\kiki\\Desktop\\training\\data\\val\\images\\'




In [28]:
def image_folder_to_array():
    image_list = []
    # Extracting all the contents in the directory corresponding to path
    l_files = os.listdir(TRAIN_IMAGE_PATH)
    # Iterating over all the files
    for file in l_files:
        file_path = f'{TRAIN_IMAGE_PATH}{file}'
            # Checking whether the given file is a directory or not
        if not os.path.isfile(file_path):
            print(f'ALERT: {file} is not a file, so can not be printed!')
        else:
            try:
                image_list.append(file_path)
            except:
                    # Catching if any error occurs and alerting the user
                print(f'ALERT: {file} could not be found! Please check\
                    the associated softwares, or the file type.')
    return image_list


img_file_arry = image_folder_to_array()
# written so notebook doesn't have to rebuild with each go.

In [29]:
def label_folder_to_array():
    label_list = []
    # Extracting all the contents in the directory corresponding to path
    # print(TRAIN_TXT_PATH)
    l_files = os.listdir(TRAIN_TXT_PATH)
    # Iterating over all the files
    for file in l_files:
        # print(file)
        # Instantiating the path of the file
        file_path = f'{TRAIN_TXT_PATH}{file}'
        # Checking whether the given file is a directory or not
        if not os.path.isfile(file_path):
            print(f'ALERT: {file} is not a file, so can not be printed!')
            break
        else:
            try:
                label_list.append(file_path)
            except:
                # Catching if any error occurs and alerting the user
                print(f'ALERT: {file} could not be found! Please check\
                the associated softwares, or the file type.')
    return label_list

In [30]:
def create_lables(img_file_arry, start, end):
    results = model(img_file_arry)  # return a list of Results objects
    # Process results list
    for i, result in enumerate(results):
        save_txt = True
        file_arry_str = img_file_arry[i]
        name = file_arry_str[start:end]
        result.save_txt(f"{TRAIN_TXT_PATH}\\{name}.txt")

In [31]:
def reclassify_lables():
    l_files = os.listdir(TRAIN_TXT_PATH)
    # Iterating over all the files
    for file in l_files:
    # Instantiating the path of the file
        file_path = f'{TRAIN_TXT_PATH}{file}'
        # Checking whether the given file is a directory or not
        if not os.path.isfile(file_path):
            print(f'ALERT: {file} is not a file, so can not be printed!')
        else:
            # Read the file
            with open(file_path, 'r') as f:
                lines = f.readlines()
            # Change line
            for i in range(len(lines)):
                if '15' in lines[i][:3]:
                    print(lines[i])
                    left = lines[i][:3]
                    right = lines[i][3:]
                    if 'mousse' in file_path:
                        left = '0'
                    else:
                        left = '1'
                    lines[i] = ' '.join([left, right])
                else:
                    lines[i] = ''
            # write the changes lines
            with open(file_path, 'w') as f:
                f.writelines(lines)


In [34]:
def clean_imgs_and_lables():
    txt_files = os.listdir(TRAIN_TXT_PATH)
    img_files = os.listdir(TRAIN_IMAGE_PATH)
    file_index = 0
    # Iterating over all the files
    while file_index < len(txt_files):
    # Instantiating the path of the file
        file_path = f"{TRAIN_IMAGE_PATH}{img_files[file_index]}"
        # Checking whether the given file is a directory or not
        if not os.path.isfile(file_path):
            print(f'ALERT: {file_path} is not a file, so can not be printed!') 
        else:
            # print(file_index)
            # find the number in both strings at index [file_index]
            img_start = img_files[file_index].find('_') + 1
            txt_start = txt_files[file_index].find('_') + 1
            img_end = img_files[file_index].find('.')
            txt_end = txt_files[file_index].find('.') 
            if txt_files[file_index][txt_start:txt_end] != img_files[file_index][img_start:img_end]:
                # if not, uhh delete img file
                print(img_files[file_index])
                file_index = 0
            # if that number is the same : pass
            file_index += 1

clean_imgs_and_lables()


In [36]:
len(img_file_arry)
len(label_file_arry)

217

In [ ]:
def move_file (file_array, destination):
    for file_source in file_array:
        dest = shutil.move(file_source, destination)

In [32]:

file_arry = image_folder_to_array()

start = file_arry[0].find('mousse')
# all files have the same path, and differ at the same point.
# this point is 42 but to avoid magic numbers we run the above.
#  
end = file_arry[0].find('.JPG') - 3
# similar to 'start' all file endings are three chars long. 
# this point is the file len - 1 (str slicing excludes the end index) 
# but to avoid magic numbers we run the above. 

# Load a model
model = YOLO("yolo11n.pt")  # pretrained YOLO11n model

label_file_arry = label_folder_to_array()
create_lables(img_file_arry, start, end)
reclassify_lables()


0: 640x640 1 cat, 1 dog, 1 bed, 60.6ms
1: 640x640 1 cat, 60.6ms
2: 640x640 1 cat, 1 bed, 60.6ms
3: 640x640 1 person, 2 cats, 60.6ms
4: 640x640 1 cat, 60.6ms
5: 640x640 1 cat, 60.6ms
6: 640x640 1 cat, 60.6ms
7: 640x640 1 cat, 1 dog, 60.6ms
8: 640x640 1 cat, 1 bed, 60.6ms
9: 640x640 1 cat, 60.6ms
10: 640x640 1 cat, 1 dog, 60.6ms
11: 640x640 2 cats, 1 dog, 1 bed, 60.6ms
12: 640x640 1 cat, 1 laptop, 1 book, 60.6ms
13: 640x640 2 persons, 1 dog, 60.6ms
14: 640x640 1 cat, 1 chair, 60.6ms
15: 640x640 3 persons, 1 dog, 1 bed, 60.6ms
16: 640x640 1 person, 1 cat, 60.6ms
17: 640x640 1 cat, 1 couch, 1 potted plant, 1 microwave, 1 oven, 1 book, 60.6ms
18: 640x640 1 cat, 60.6ms
19: 640x640 1 cat, 2 bears, 1 book, 60.6ms
20: 640x640 1 cat, 1 chair, 60.6ms
21: 640x640 1 person, 1 cat, 60.6ms
22: 640x640 2 cats, 60.6ms
23: 640x640 1 dog, 1 couch, 60.6ms
24: 640x640 1 cat, 1 dog, 1 chair, 1 couch, 4 books, 60.6ms
25: 640x640 1 dog, 60.6ms
26: 640x640 1 cat, 1 bear, 60.6ms
27: 640x640 1 cat, 1 dog, 60.6m

In [ ]:

# now to randomly sort and move imgs and txt files into /train or /val
X_train, X_test, y_train, y_test = train_test_split(
     img_file_arry, label_file_arry, test_size=0.2, random_state=42)

# x = imgs
# y = lables

move_file(X_test, VAL_IMAGE_PATH) 
move_file(y_test, VAL_TXT_PATH)

# Train the model
results = model.train(data="cat_data.yaml", epochs=100, imgsz=640)

In [ ]:
model.predict('C:/Users/kiki/Desktop/training/sherbert/sherbert_103.jpg')

In [ ]:
# Load a model
model = YOLO("yolo11n.pt")  # pretrained YOLO11n model
results = model.train(data="cat_data.yaml", epochs=100, imgsz=640)

now we have to figure out a way to place diffrent class # in our txt file.
0 = M
1 = Sh

right now, 15 = cat. 

when we train the model, looks like we'll want a confidence of 70 

In [34]:
start = file_arry[0].find('mousse')
# all files have the same path, and differ at the same point. this point is 42 but to avoid magic numbers we run the above. 
end = file_arry[0].find('.JPG')-3
string = file_arry[200]
string[start:end]

'sherbert_44'

In [ ]:
from pathlib import Path

def rename_files(folder_path, prefix="", suffix="", start_number=1, keep_extension=True):
    """
    Rename all files in a folder with a pattern.
    
    Args:
        folder_path: Path to the folder containing files
        prefix: Text to add before the number (e.g., "image_")
        suffix: Text to add after the number (e.g., "_photo")
        start_number: Starting number for sequential numbering
        keep_extension: Whether to keep original file extensions
    """
    folder = Path(folder_path)
    
    if not folder.exists():
        print(f"Error: Folder '{folder_path}' does not exist")
        return
    
    # Get all files (not directories) in the folder
    files = [f for f in folder.iterdir() if f.is_file()]
    
    if not files:
        print("No files found in the folder")
        return
    
    # Sort files by name for consistent ordering
    files.sort()
    
    print(f"Found {len(files)} files to rename\n")
    # Perform renaming
    for i, file in enumerate(files, start=start_number):
        extension = file.suffix if keep_extension else ""
        new_name = f"{prefix}{i}{suffix}{extension}"
        new_path = folder / new_name
        
        try:
            file.rename(new_path)
            print(f"✓ Renamed: {file.name} -> {new_name}")
        except Exception as e:
            print(f"✗ Error renaming {file.name}: {e}")
    
    print("\nRenaming complete!")


# Example usage:
if __name__ == "__main__":
    # Option 1: Basic sequential numbering (file1.txt, file2.txt, etc.)
    rename_files(
        folder_path=f'C:\\Users\\kiki\\Desktop\\training\\data\\train\\images\\',
        prefix="mousse_",
        start_number=1
    )

Found 282 files to rename

✓ Renamed: crop_1.jpg -> m_1.jpg
✓ Renamed: crop_110.jpg -> m_2.jpg
✓ Renamed: crop_1100.jpg -> m_3.jpg
✓ Renamed: crop_1101.jpg -> m_4.jpg
✓ Renamed: crop_1102.jpg -> m_5.jpg
✓ Renamed: crop_1103.jpg -> m_6.jpg
✓ Renamed: crop_1104.jpg -> m_7.jpg
✓ Renamed: crop_1105.jpg -> m_8.jpg
✓ Renamed: crop_1106.jpg -> m_9.jpg
✓ Renamed: crop_1107.jpg -> m_10.jpg
✓ Renamed: crop_1108.jpg -> m_11.jpg
✓ Renamed: crop_1109.jpg -> m_12.jpg
✓ Renamed: crop_111.jpg -> m_13.jpg
✓ Renamed: crop_1110.jpg -> m_14.jpg
✓ Renamed: crop_1111.jpg -> m_15.jpg
✓ Renamed: crop_1112.jpg -> m_16.jpg
✓ Renamed: crop_1113.jpg -> m_17.jpg
✓ Renamed: crop_1114.jpg -> m_18.jpg
✓ Renamed: crop_1115.jpg -> m_19.jpg
✓ Renamed: crop_1116.jpg -> m_20.jpg
✓ Renamed: crop_1117.jpg -> m_21.jpg
✓ Renamed: crop_1118.jpg -> m_22.jpg
✓ Renamed: crop_1119.jpg -> m_23.jpg
✓ Renamed: crop_112.jpg -> m_24.jpg
✓ Renamed: crop_1120.jpg -> m_25.jpg
✓ Renamed: crop_1121.jpg -> m_26.jpg
✓ Renamed: crop_1122.jpg -

In [ ]:
def image_cropper(name):
    path = f'C:\\Users\\kiki\\Desktop\\training\\{name}'
    # Extracting all the contents in the directory corresponding to path
    l_files = os.listdir(path)

    # Iterating over all the files
    for file in l_files:

    # Instantiating the path of the file
        file_path = f'{path}\\{file}'

        # Checking whether the given file is a directory or not
        if os.path.isfile(file_path):
            try:
                # crop file
                # print(f'Cropping {file}')

                cropper = solutions.ObjectCropper(
                    show=True,  # display the output
                    model="yolo11n.pt",  # model for object cropping, e.g., yolo11x.pt.
                    classes=[15],  # crop objs identified as cats
                    conf=0.2,  # adjust confidence threshold for the objects.
                    crop_dir=f"C:\\Users\\kiki\\Desktop\\training\\cleaned\\{name}_cleaned" # set the directory name for cropped detections
                )
                frame = cv2.imread(file_path)
                processed_results = cropper.process(frame) #saves to directory

                time.sleep(2) 
            except:
                # Catching if any error occurs and alerting the user
                print(f'ALERT: {file} could not be cropped! Please check\
                the associated softwares, or the file type.')
        else:
            print(f'ALERT: {file} is not a file, so can not be printed!')

        # print('Task finished!')




In [ ]:
for cat in cat_names:
    image_cropper(cat)

# what if instead of cropping, i spit out the region it detects and save that to a .txt file with the same name as the given img 

In [10]:
# using clip to encode the cleaned imgs into vectors
# model only accepts tensors. we will use pytorch to transform out img
# into a torch tensor

# need to refactor to save all data into a single array and then append single array to img_arry 
def Img_to_tensor(name, path_end):
    # to first do this we need to create a transform
    # as a note this should only be done once
    temp_numpy = []
    temp_is_mousse = []
    transform = transforms.Compose([
        transforms.PILToTensor()
    ])
    path = f'C:\\Users\\kiki\\Desktop\\training\\cleaned\\{name}{path_end}'
    l_files = os.listdir(path)

    # Iterating over all the files
    for file in l_files:
    # Instantiating the path of the file
        file_path = f'{path}\\{file}'
        # Checking whether the given file is a directory or not
        if not os.path.isfile(file_path):
            print(f'ALERT: {file_path} could not be transformed! Please check\
            the associated softwares, or the file type.')
        else:
            # crop file
            #print(f'transforming {file}')
            image = clip_preprocess(Image.open(file_path)).unsqueeze(0).to(device)
            with torch.no_grad():
                image_features = clip_model.encode_image(image)
                image_features = image_features / image_features.norm(dim=-1, keepdim=True)

            #print(f"Image encoding shape: {image_features.shape}")
            #print(f"Image object: {image_features} {type(image_features)}")
        temp_numpy.append(image_features.numpy())
        temp_is_mousse.append(cat == 'mousse')

    return temp_numpy, temp_is_mousse



In [11]:
# used to store resulting, unscrambled tensor array and their bool val 
img_array = []
img_array_is_mousse = []

if USE_HUMAN_CLEANED_IMGS:
    path_end = "_cleaned_done"
else:
    path_end = ""

for cat in cat_names:
    cat_vector , is_mousse_vector = Img_to_tensor(cat, path_end)
    img_array += cat_vector
    img_array_is_mousse += is_mousse_vector

In [ ]:
# data wrangle
img_array = [x.flatten() for x in img_array]

In [17]:
img_array

[array([  0.0062857,    0.022423,    0.016246, ...,   -0.004739,   -0.002528, -0.00070693], shape=(1024,), dtype=float32),
 array([   0.003883,    0.014323,  -0.0059982, ...,   -0.003721,   0.0093456,    0.020672], shape=(1024,), dtype=float32),
 array([ -0.0069074,    0.013334,   -0.016574, ...,  -0.0066198,    0.034125,    0.027107], shape=(1024,), dtype=float32),
 array([  -0.013148,   0.0094255,  0.00041086, ...,  -0.0066611,  -0.0081412,    0.049442], shape=(1024,), dtype=float32),
 array([  0.0072935,   0.0070948,  0.00094771, ...,  -0.0025397,    0.013511,    0.049459], shape=(1024,), dtype=float32),
 array([  0.0030974,    0.013215, -0.00024867, ...,   0.0045428,    0.007836,     0.07035], shape=(1024,), dtype=float32),
 array([-0.00082921,    0.022509,   -0.010633, ..., -0.00096478,    0.031053,    0.027658], shape=(1024,), dtype=float32),
 array([   0.021762,    0.017487,    0.029768, ...,   -0.014067,    0.022328,    0.090751], shape=(1024,), dtype=float32),
 array([  -0.016

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
     img_array, img_array_is_mousse, test_size=0.2, random_state=42)

print(X_train) 
print(X_test) 
print(y_train) 
print(y_test)

[array([-0.00028044,    0.017444,    -0.02812, ...,  -0.0089982,  -0.0054974,    0.045279], shape=(1024,), dtype=float32), array([  -0.020098,    0.025196,   -0.012369, ...,   -0.008528,    0.014467,    0.042082], shape=(1024,), dtype=float32), array([ 5.7576e-06,    0.037587,  -0.0065563, ...,   -0.010607,    0.025061,    0.013197], shape=(1024,), dtype=float32), array([  -0.020367,    0.020078,   -0.017627, ...,  -0.0089056,    0.021586,    0.036556], shape=(1024,), dtype=float32), array([  -0.020741,    0.019555,   -0.011011, ...,  -0.0036634,   0.0070865,    0.060652], shape=(1024,), dtype=float32), array([ -0.0023339,     0.01617,    -0.03016, ...,   0.0087698,      0.0296,    0.040298], shape=(1024,), dtype=float32), array([  0.0026065,    0.018088,    0.038183, ...,   -0.033905,    0.003017,     0.05384], shape=(1024,), dtype=float32), array([ -0.0045734,    0.014292,  -0.0061087, ...,   -0.016232,  -0.0011962,    0.040553], shape=(1024,), dtype=float32), array([ -0.0065462,    

In [19]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [ ]:
y_predict = clf.predict(X_test)

In [ ]:
total_values = len(y_test)
correct_values = 0

for predict,actual in zip(y_predict, y_test):
    correct_values += 1 if predict == actual else 0

acc = (correct_values / total_values) * 100
print(acc)

# play around with other models, look into convolution/convoling + data comphresion 
# look into smaller output array for encodings

71.69811320754717


In [ ]:


#test code snippet for cropping
test_img = "C:\\Users\\kiki\\Desktop\\training\\mousse\\mousse_10.jpg"

cropper = solutions.ObjectCropper(
    show=True,  # display the output
    model="yolo11n.pt",  # model for object cropping, e.g., yolo11x.pt.
    classes=[15],  # crop objs identified as cats
    # conf=0.5,  # adjust confidence threshold for the objects.
    crop_dir="C:\\Users\\kiki\\Desktop\\training\\cleaned\\mousse_cleaned" # set the directory name for cropped detections
)

frame = cv2.imread(test_img)
processed_results = cropper.process(frame) #saves to directory


Ultralytics Solutions:  {'source': None, 'model': 'yolo11n.pt', 'classes': [15], 'show_conf': True, 'show_labels': True, 'region': None, 'colormap': 21, 'show_in': True, 'show_out': True, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'figsize': (12.8, 7.2), 'blur_ratio': 0.5, 'vision_point': (20, 20), 'crop_dir': 'C:\\Users\\kiki\\Desktop\\training\\cleaned\\mousse_cleaned', 'json_file': None, 'line_width': 2, 'records': 5, 'fps': 30.0, 'max_hist': 5, 'meter_per_pixel': 0.05, 'max_speed': 120, 'show': True, 'iou': 0.7, 'conf': 0.25, 'device': None, 'max_det': 300, 'half': False, 'tracker': 'botsort.yaml', 'verbose': True, 'data': 'images'}
WARNING show=True disabled for crop solution, results will be saved in the directory named: C:\Users\kiki\Desktop\training\cleaned\mousse_cleaned


In [5]:
#test snippet for seeing confidence

test_img = "C:\\Users\\kiki\\Desktop\\training\\mousse\\mousse_105.jpg"
model= YOLO("yolo11n.pt")
results = model.predict(
  test_img,
  classes=[15])

results[0].show()  # Display results


image 1/1 C:\Users\kiki\Desktop\training\mousse\mousse_105.jpg: 480x640 3 cats, 73.9ms
Speed: 4.0ms preprocess, 73.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
